In [ ]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
from scipy import ndimage
from nilearn.image import resample_to_img

# ==========================================
# 1. CONFIGURATION (FILL IN PATHS HERE)
# ==========================================

# --- File Paths ---
# Path to the ACAI map (e.g., ACAI_native_....nii)
FILEPATH_ACAI = r""

# Path to the Neuromorphometrics Atlas of the patient(neuromorphometrics_...nii)
# OPTIONAL: Set to None or "" to skip atlas masking and ROI reporting.
FILEPATH_ATLAS = None

# --- Parameters ---
ACAI_THRESHOLD = -19.0        # Voxels must be LOWER than this (e.g., -15 or -19)
MIN_CLUSTER_SIZE_MM3 = 2000   # Minimum cluster size in mm³ (e.g. 2.0 cm³)

# --- Derived output paths & filenames ---
patient_dir = os.path.dirname(os.path.abspath(FILEPATH_ACAI)) if FILEPATH_ACAI else os.getcwd()

# Output folder name based on input filename (without extension)
acai_filename_no_ext = os.path.splitext(os.path.basename(FILEPATH_ACAI))[0]
OUTPUT_DIR = os.path.join(patient_dir, acai_filename_no_ext)

def _format_value_label(value):
    return f"{value:.3f}".rstrip('0').rstrip('.')

cluster_size_cm3_label = _format_value_label(MIN_CLUSTER_SIZE_MM3 / 1000.0)
abs_threshold_label = _format_value_label(abs(ACAI_THRESHOLD))

# Excel filename: Clusters_{threshold}_{size}cm3.xlsx
FILENAME_EXCEL = f"Clusters_{abs_threshold_label}_{cluster_size_cm3_label}cm3.xlsx"

In [ ]:
# ==========================================
# 2. ROI DICTIONARY & LISTS
# ==========================================

# Full mapping of Neuromorphometrics ID -> Region Name
ROI_MAP = {
    # Left Hemisphere
    7: "Left Amygdala", 21: "Left Hippocampus", 45: "Left Anterior Cingulate Gyrus", 
    47: "Left Anterior Insula", 49: "Left Anterior Orbital Gyrus", 51: "Left Angular Gyrus",
    53: "Left Calcarine and Cerebrum", 55: "Left Central Operculum", 57: "Left Cuneus", 
    59: "Left Entorhinal Area", 61: "Left Frontal Operculum", 63: "Left Frontal Pole", 
    65: "Left Fusiform Gyrus", 67: "Left Gyrus Rectus", 69: "Left Inferior Occipital Gyrus", 
    71: "Left Inferior Temporal Gyrus", 73: "Left Lingual Gyrus", 75: "Left Lateral Orbital Gyrus", 
    77: "Left Middle Cingulate Gyrus", 79: "Left Medial Frontal Cerebrum", 81: "Left Middle Frontal Gyrus", 
    83: "Left Middle Occipital Gyrus", 85: "Left Medial Orbital Gyrus", 87: "Left Medial Postcentral Gyrus", 
    89: "Left Medial Precentral Gyrus", 91: "Left Superior Medial Frontal Gyrus", 93: "Left Middle Temporal Gyrus", 
    95: "Left Occipital Pole", 97: "Left Occipital Fusiform Gyrus", 99: "Left Inferior Frontal Gyrus", 
    101: "Left Inferior Frontal Orbital Gyrus", 103: "Left Posterior Cingulate Gyrus", 105: "Left Precuneus", 
    107: "Left Parahippocampus Gyrus", 109: "Left Posterior Insula", 111: "Left Parietal Operculum", 
    113: "Left Postcentral Gyrus", 115: "Left Posterior Orbital Gyrus", 117: "Left Planum Polare", 
    119: "Left Precentral Gyrus", 121: "Left Temporal", 125: "Left Superior Frontal Gyrus", 
    127: "Left Cerebrum and Motor", 129: "Left Supramarginal Gyrus", 131: "Left Superior Occipital Gyrus", 
    133: "Left Superior Parietal Lobule", 135: "Left Superior Temporal Gyrus", 137: "Left Temporal Pole", 
    139: "Left Inferior Frontal Angular Gyrus", 141: "Left Temporal Transverse Gyrus",

    # Right Hemisphere
    8: "Right Amygdala", 22: "Right Hippocampus", 46: "Right Anterior Cingulate Gyrus", 
    48: "Right Anterior Insula", 50: "Right Anterior Orbital Gyrus", 52: "Right Angular Gyrus", 
    54: "Right Calcarine and Cerebrum", 56: "Right Central Operculum", 58: "Right Cuneus", 
    60: "Right Entorhinal Area", 62: "Right Frontal Operculum", 64: "Right Frontal Pole", 
    66: "Right Fusiform Gyrus", 68: "Right Gyrus Rectus", 70: "Right Inferior Occipital Gyrus", 
    72: "Right Inferior Temporal Gyrus", 74: "Right Lingual Gyrus", 76: "Right Lateral Orbital Gyrus", 
    78: "Right Middle Cingulate Gyrus", 80: "Right Medial Frontal Cerebrum", 82: "Right Middle Frontal Gyrus", 
    84: "Right Middle Occipital Gyrus", 86: "Right Medial Orbital Gyrus", 88: "Right Medial Postcentral Gyrus", 
    90: "Right Medial Precentral Gyrus", 92: "Right Superior Medial Frontal Gyrus", 94: "Right Middle Temporal Gyrus", 
    96: "Right Occipital Pole", 98: "Right Occipital Fusiform Gyrus", 100: "Right Inferior Frontal Gyrus", 
    102: "Right Inferior Frontal Orbital Gyrus", 104: "Right Posterior Cingulate Gyrus", 106: "Right Precuneus", 
    108: "Right Parahippocampus Gyrus", 110: "Right Posterior Insula", 112: "Right Parietal Operculum", 
    114: "Right Postcentral Gyrus", 116: "Right Posterior Orbital Gyrus", 118: "Right Planum Polare", 
    120: "Right Precentral Gyrus", 122: "Right Temporal", 126: "Right Superior Frontal Gyrus", 
    128: "Right Cerebrum and Motor", 130: "Right Supramarginal Gyrus", 132: "Right Superior Occipital Gyrus", 
    134: "Right Superior Parietal Lobule", 136: "Right Superior Temporal Gyrus", 138: "Right Temporal Pole", 
    140: "Right Inferior Frontal Angular Gyrus", 142: "Right Temporal Transverse Gyrus"
}

# Create the list of Cortical IDs from the keys of the map
ALL_CORTICAL_IDS = list(ROI_MAP.keys())
print(f"Loaded {len(ALL_CORTICAL_IDS)} Cortical ROIs.")

In [ ]:
# ==========================================
# 3. RUN ANALYSIS
# ==========================================

def process_patient():
    print("--- Starting Processing ---")
    
    # 1. Load ACAI (Primary Image)
    if not os.path.exists(FILEPATH_ACAI):
        raise FileNotFoundError(f"ACAI missing: {FILEPATH_ACAI}")
    img_acai = nib.load(FILEPATH_ACAI)
    data_acai = img_acai.get_fdata()
    affine_ref = img_acai.affine
    
    # Calc voxel volume
    vox_vol = np.prod(img_acai.header.get_zooms()[:3])
    print(f"Voxel Volume: {vox_vol:.3f} mm³")
    
    # 2. Load Atlas (Optional)
    use_atlas = False
    data_atlas = None
    
    if FILEPATH_ATLAS and os.path.exists(FILEPATH_ATLAS):
        print(f"Loading Atlas: {FILEPATH_ATLAS}")
        img_atlas = nib.load(FILEPATH_ATLAS)
        
        # Check dimensions against ACAI
        if img_atlas.shape != img_acai.shape:
            print(f"Shape mismatch! ACAI:{img_acai.shape} vs Atlas:{img_atlas.shape}. Resampling Atlas to ACAI space...")
            img_atlas = resample_to_img(img_atlas, img_acai, interpolation='nearest')
        
        data_atlas = img_atlas.get_fdata().astype(int)
        use_atlas = True
    else:
        print("Atlas not provided or not found. Skipping Atlas masking and ROI identification.")

    # 3. Create Analysis Mask
    # Threshold ACAI (ACAI <= Threshold)
    mask_thresh = (data_acai <= ACAI_THRESHOLD)
    
    if use_atlas:
        # Filter by Cortical ROIs if atlas is present
        mask_cortex = np.isin(data_atlas, ALL_CORTICAL_IDS)
        mask_final = mask_thresh & mask_cortex
        print(f"Applying Atlas Mask. Voxels before: {np.sum(mask_thresh)}, after: {np.sum(mask_final)}")
    else:
        mask_final = mask_thresh
        print(f"No Atlas Mask applied. Voxels: {np.sum(mask_final)}")
    
    if np.sum(mask_final) == 0:
        print("ERROR: Mask is empty.")
        return
    
    labeled_array, _ = ndimage.label(mask_final)
    
    # 4. Analyze Clusters & Build Excel Data
    excel_rows = []
    
    # Initialize map for ALL clusters
    final_map = np.zeros_like(labeled_array, dtype=np.uint16)
    
    # Get unique labels (excluding background 0)
    labels = np.unique(labeled_array)
    labels = labels[labels != 0]
    
    # Ensure output directory exists
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"Created output directory: {OUTPUT_DIR}")
    
    cluster_counter = 1
    
    for lbl in labels:
        cluster_mask = labeled_array == lbl
        size_mm3 = np.sum(cluster_mask) * vox_vol
        
        if size_mm3 >= MIN_CLUSTER_SIZE_MM3:
            # Add to final combined map
            final_map[cluster_mask] = cluster_counter
            
            # Save individual cluster NIfTI
            # Create a binary mask for this cluster (1s and 0s)
            single_cluster_data = np.zeros_like(labeled_array, dtype=np.uint8)
            single_cluster_data[cluster_mask] = 1
            
            # Construct filename: cluster_1_19_2cm3.nii
            cluster_filename = f"cluster_{cluster_counter}_{abs_threshold_label}_{cluster_size_cm3_label}cm3.nii"
            cluster_filepath = os.path.join(OUTPUT_DIR, cluster_filename)
            
            nib.save(nib.Nifti1Image(single_cluster_data, affine_ref), cluster_filepath)
            
            # Find ROIs in this cluster (if atlas used)
            roi_ids_str = ""
            roi_names_str = ""
            num_rois = 0
            
            if use_atlas:
                rois_in_cluster = data_atlas[cluster_mask]
                unique_rois = np.unique(rois_in_cluster)
                
                # Filter for valid cortical IDs and sort
                valid_rois = sorted([r for r in unique_rois if r in ROI_MAP])
                
                roi_ids_str = ", ".join(map(str, valid_rois))
                roi_names_str = "; ".join([ROI_MAP[r] for r in valid_rois])
                num_rois = len(valid_rois)
            
            # Add to list
            excel_rows.append({
                "Cluster_Number": cluster_counter,
                "Cluster_Size_cm3": size_mm3 / 1000.0,
                "Number_of_ROIs": num_rois,
                "ROI_IDs": roi_ids_str,
                "ROI_Names": roi_names_str,
                "Filename": cluster_filename
            })
            cluster_counter += 1
    
    # 5. Save Results
    
    # Save Combined NIfTI
    if cluster_counter > 1: 
        combined_filename = f"All_Clusters_{abs_threshold_label}_{cluster_size_cm3_label}cm3.nii"
        combined_filepath = os.path.join(OUTPUT_DIR, combined_filename)
        nib.save(nib.Nifti1Image(final_map, affine_ref), combined_filepath)
        print(f"Saved Combined NIfTI: {combined_filepath}")
        
    # Save Excel
    out_xls = os.path.join(OUTPUT_DIR, FILENAME_EXCEL)
    if excel_rows:
        df = pd.DataFrame(excel_rows)
        df["Cluster_Size_cm3"] = df["Cluster_Size_cm3"].map(lambda v: f"{v:.2f}")
        
        df.to_excel(out_xls, index=False)
        print(f"Saved Excel: {out_xls}")
        print(f"Saved {len(excel_rows)} cluster NIfTI files in {OUTPUT_DIR}")
        print("\nTop 5 Clusters:")
        print(df.head())
    else:
        print("No clusters found matching size criteria. No Excel or NIfTI files saved.")

# Run
process_patient()